In [ ]:
# !pip install torch_geometric


In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

import torch
import torch_geometric
import torch.nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn import Linear, LayerNorm, ReLU, Dropout
from torch_geometric.nn import ChebConv, NNConv, DeepGCNLayer, GATConv, DenseGCNConv, GCNConv, GraphConv, MessagePassing
from torch_geometric.data import Data, DataLoader
from torch_geometric.utils import add_self_loops, degree
import torch_geometric.transforms as T


from sklearn.metrics import roc_auc_score
import scipy.sparse as sp

import warnings
warnings.filterwarnings("ignore")

In [32]:
print(torch.__version__)

1.12.1


In [33]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print(device)


cpu


In [34]:
# datasets_folder = '/content/drive/MyDrive/Colab Notebooks/ssn/projekt/datasets/elliptic_augmented_dataset/'
datasets_folder = '../datasets/elliptic_augmented_dataset/'

In [35]:
def split_edges_into_train_and_test(edges_dataframe, node_features_with_class):
    edges_amount = len(edges_dataframe)
    step = int(edges_amount * 0.02)    

    train_edges = [] # edges which have both nodes labeled.
    test_edges = []  # edges which have both nodes unlabeled.
    
    # this may take a long time
    for i, edge in enumerate(np.array(edges_dataframe)):
        if i % step == 0:
            print(f'{ round(i/ edges_amount * 100)}%')
            
        found_start_node = node_features_with_class[node_features_with_class[0] == edge[0]]
        
        #ignore edges for which a node does not exist (this happened once)
        if len(found_start_node) == 0: continue
            
        found_end_node = node_features_with_class[node_features_with_class[0] == edge[1]]
        if len(found_end_node) == 0: continue
            
        if found_start_node.values[0, -1] != 2 and found_end_node.values[0, -1] != 2:
            train_edges.append(edge)
            continue
        
        if found_start_node.values[0, -1] == 2 and found_end_node.values[0, -1] == 2:
            test_edges.append(edge)
            continue
          
        # if found_start_node.values[0, -1] == 2:
        #   test_edges.append(edge)
        #   continue
        
        # if found_end_node.values[0, -1] == 2:
        #   test_edges.append(edge)
        # else:
        #   train_edges.append(edge)
          
    train_edges = pd.DataFrame(train_edges)
    test_edges = pd.DataFrame(test_edges)
    return train_edges, test_edges

In [36]:
# This will read from cached files if they exist. If you change the code, delete the cached files.
def split_edges_into_train_and_test_IO(datasets_folder):
    train_edges_filepath = datasets_folder + 'train_edges.csv'
    test_edges_filepath = datasets_folder + 'test_edges.csv'
    
    train_edges = [] # edges which have both nodes labeled.
    test_edges = [] # edges which one or both nodes are unlabeled.
    if os.path.isfile(train_edges_filepath) and os.path.isfile(test_edges_filepath):
        train_edges = pd.read_csv(train_edges_filepath, sep=',').drop(columns=['Unnamed: 0']).rename(columns = {'0': 0, '1':1})
        test_edges = pd.read_csv(test_edges_filepath, sep=',').drop(columns=['Unnamed: 0']).rename(columns = {'0': 0, '1':1})
        
    else:
        edges_filepath = datasets_folder + 'elliptic_txs_edgelist.csv'
        edges_dataframe = pd.read_csv(edges_filepath, sep=',')
        train_edges, test_edges = split_edges_into_train_and_test(edges_dataframe, node_features_dataframe_with_class_without_timestep)
        
        # save the train_edges and test_edges to file so that the expensive split does not have to run every time.
        train_edges.to_csv(train_edges_filepath)
        test_edges.to_csv(test_edges_filepath)
    
    return train_edges, test_edges

In [37]:
classes_filepath = datasets_folder + 'elliptic_txs_classes.csv'
classes_dataframe = pd.read_csv(classes_filepath, sep=',')
classes_dataframe

,txId,class
0,230425980,unknown
1,5530458,unknown
2,232022460,unknown
3,232438397,2
4,230460314,unknown
...,...,...
276700,-80255194,unknown
276701,-229310245,suspicious
276702,-223754303,unknown
276703,-625924180,suspicious


In [38]:
# initially 2 is licit, 1 is illicit, suspicious is also illicit, unknown is unknown
# after remapping 0 is illicit, 1 is licit, 2 is unknown
# rows with class 0 and 1 will be used for training
# rows with class 2 will be used for prediction

def remap_label(label):
  if label == '1': return 0
  if label == 'suspicious': return 0
  if label == '2': return 1
  return 2

In [39]:
classes_dataframe_after_remap = classes_dataframe.copy()
classes_dataframe_after_remap['class'] = classes_dataframe['class'].map(remap_label)
classes_dataframe_after_remap

,txId,class
0,230425980,2
1,5530458,2
2,232022460,2
3,232438397,1
4,230460314,2
...,...,...
276700,-80255194,2
276701,-229310245,0
276702,-223754303,2
276703,-625924180,0


In [40]:
node_features_filepath = datasets_folder + 'elliptic_txs_features.csv'
node_features_dataframe = pd.read_csv(node_features_filepath, sep=',', header=None)
# node_features_dataframe
node_features_dataframe_with_class = node_features_dataframe.copy()
node_features_dataframe_with_class['class'] = classes_dataframe_after_remap['class']
# node_features_dataframe_with_class
node_features_dataframe_with_class_without_timestep = node_features_dataframe_with_class.drop(columns=[1])
node_features_dataframe_with_class_without_timestep

,0,2,3,4,5,6,7,8,9,10,...,158,159,160,161,162,163,164,165,166,class
0,230425980,-0.056189,-0.175567,-1.185821,-0.186390,-0.046514,-0.167505,-0.101009,-0.133295,-0.088508,...,-0.582752,1.442895,1.440189,0.003514,-0.092606,-0.135784,-0.101068,-0.102834,-0.102128,2
1,5530458,-0.056193,-0.175567,-1.185821,-0.186390,-0.046514,-0.167505,-0.101009,-0.133311,-0.088517,...,0.702410,-1.033609,-1.032884,0.003514,-0.092606,-0.135784,-0.101068,-0.102834,-0.102128,2
2,232022460,-0.056378,-0.175567,-1.185821,-0.186390,-0.046514,-0.167505,-0.101009,-0.133972,-0.088883,...,0.467009,-1.033609,-1.032884,-0.106566,-0.113936,-0.135784,-0.186659,-0.102834,-0.102128,2
3,232438397,0.043005,0.743641,-0.644127,2.874647,-0.068726,2.304311,3.142835,-0.134902,-0.058112,...,-0.595477,0.204643,0.203652,0.994235,0.099367,-0.135784,0.669253,-0.102834,-0.102128,1
4,230460314,0.294595,-0.131268,-1.185821,0.125213,0.375512,0.188604,-0.101009,-0.134776,0.033618,...,-0.380435,0.484856,0.546219,0.003514,0.312670,0.369103,1.281230,0.201809,0.202687,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276700,-80255194,-0.015471,0.043889,1.522650,-0.058083,-0.046514,-0.020871,-0.101009,-0.134902,-0.015154,...,-0.302816,1.442895,1.440189,0.003514,-0.092606,-0.135784,-0.101068,-0.102834,-0.102128,2
276701,-229310245,-0.056634,-0.141863,0.980956,-0.149731,-0.068726,-0.125609,-0.101009,-0.134903,-0.089391,...,-0.570028,0.204643,0.203652,-0.106566,-0.113936,-0.135784,-0.186659,-0.102834,-0.102128,0
276702,-223754303,-0.056336,-0.119014,0.980956,-0.149731,-0.068726,-0.125609,-0.101009,-0.134890,-0.088828,...,-0.595477,0.204643,0.203652,0.003514,-0.092606,-0.135784,-0.101068,-0.102834,-0.102128,2
276703,-625924180,-0.052898,-0.109119,-1.185821,-0.131401,-0.068726,-0.167505,0.136346,-0.134233,-0.085575,...,0.447922,0.204643,0.203652,-0.106566,-0.113936,-0.135784,-0.186659,-0.102834,-0.102128,0


In [41]:
train_edges, test_edges = split_edges_into_train_and_test_IO(datasets_folder)
print(len(train_edges))
print(len(test_edges))
train_edges

36633
131778


,0,1
0,232344069,27553029
1,3881097,232457116
2,232051089,232470704
3,230473487,7089694
4,231182296,14660781
...,...,...
36628,-335492958,336712245
36629,-16565051,21440143
36630,-335493020,336712245
36631,-335493021,336712245


In [42]:
# Splitting nodes into train and test
train_node_features = node_features_dataframe_with_class_without_timestep[node_features_dataframe_with_class_without_timestep['class'] != 2]
test_node_features = node_features_dataframe_with_class_without_timestep[node_features_dataframe_with_class_without_timestep['class'] == 2]
# train_node_features
train_node_features_without_class = train_node_features.drop(columns=['class'])
test_node_features_without_class = test_node_features.drop(columns=['class'])

In [43]:
def create_reindex_dict(node_features_without_class): 
    reindex_dictionary = dict()
    for i, node_id in enumerate(node_features_without_class[0]):
      reindex_dictionary[node_id] = i
    return reindex_dictionary

In [44]:
# # now we need to reindex the train nodes and train edges so that node id is equal to row id
def reindex_edges(node_features_without_class, edges):
    reindex_dictionary = create_reindex_dict(node_features_without_class)
    edges_reindexed = edges.copy()
    edges_reindexed[0] = edges[0].map(lambda node_id: reindex_dictionary[node_id])
    edges_reindexed[1] = edges[1].map(lambda node_id: reindex_dictionary[node_id])
    return edges_reindexed


In [45]:
train_edges_reindexed = reindex_edges(train_node_features_without_class, train_edges)


In [46]:
# reindex_dictionary = create_reindex_dict(test_node_features_without_class)
# test_edges[0].map(lambda node_id: reindex_dictionary[node_id])

In [47]:
test_edges_reindexed = reindex_edges(test_node_features_without_class, test_edges)
# now we have test nodes & test edges which can be used to test the trained model

In [48]:
def create_train_data_object(train_edges_reindexed, train_node_features_without_class, train_classes):
    edge_index = torch.tensor(np.array(train_edges_reindexed), dtype=torch.long)
    x = torch.tensor(np.array(train_node_features_without_class), dtype=torch.float)
    y = torch.tensor(np.array(train_classes), dtype=torch.float)
    data = Data(x=x, edge_index=edge_index.t().contiguous(), y=y)
    data.validate(raise_on_error = True)
    split = T.RandomNodeSplit(num_val=0.1, num_test=0)
    data_with_masks = split(data)
    return data_with_masks

In [49]:
def create_test_data_object(test_edges_reindexed, test_node_features_without_class):
    edge_index = torch.tensor(np.array(test_edges_reindexed), dtype=torch.long)
    x = torch.tensor(np.array(test_node_features_without_class), dtype=torch.float)
    data = Data(x=x, edge_index=edge_index.t().contiguous())
    data.validate(raise_on_error = True)
    return data

In [50]:
data_with_masks = create_train_data_object(train_edges_reindexed, train_node_features_without_class, train_node_features['class'])
data_with_masks

Data(x=[71496, 166], edge_index=[2, 36633], y=[71496], train_mask=[71496], val_mask=[71496], test_mask=[71496])

In [51]:
test_data_object = create_test_data_object(test_edges_reindexed, test_node_features_without_class)
test_data_object

Data(x=[205209, 166], edge_index=[2, 131778])

In [52]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.hid = 8
        self.in_head = 8
        self.out_head = 1
        self.conv1 = GATConv(data_with_masks.num_features, self.hid, heads = self.in_head)
        self.conv2 = GATConv(self.hid * self.in_head, 1, concat=False, heads = self.out_head)

    def forward(self, x, edge_index):
        x = F.dropout(x, p=0.6, training = self.training)
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.6, training = self.training)
        x = self.conv2(x, edge_index)
        return F.sigmoid(x)

In [53]:
model = Net().to(device)
data_in_device = data_with_masks.to(device)

In [54]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')
criterion = torch.nn.BCELoss()

In [55]:
model.train()
for epoch in range(170):
    optimizer.zero_grad()
    out = model(data_in_device.x, data_in_device.edge_index)
    out = out.reshape((data_in_device.x.shape[0]))
    loss = criterion(out[data_in_device.train_mask], data_in_device.y[data.train_mask])
    auc = roc_auc_score(data_in_device.y.detach().cpu().numpy(), out.detach().cpu().numpy()) #[train_idx]

    if epoch%5 == 0:
        train_auc = roc_auc_score(data.y[data_in_device.train_mask].detach().cpu().numpy(), out[data_in_device.train_mask].detach().cpu().numpy()) 
        val_auc = roc_auc_score(data.y[data_in_device.val_mask].detach().cpu().numpy(), out[data_in_device.val_mask].detach().cpu().numpy())
        print("epoch: {} - loss: {} - train_roc: {} - val_auc: {}".format(epoch, loss.item(), train_auc, val_auc))
    loss.backward()
    optimizer.step()

model.eval()

NameError: name 'data' is not defined

In [ ]:
# now testing the model on unlabeled dataset to see if proportions between predicted classes are as expected.

In [ ]:
data_in_device = test_data_object.to(device)
output = model(data_in_device.x, data_in_device.edge_index)
output


In [ ]:
amount_of_all_test_samples = output.shape[0]
amount_of_licit_predicted_transactions = output.count_nonzero().item()
amount_of_illicit_predicted_transactions = amount_of_all_test_samples - amount_of_licit_predicted_transactions

print(f'{amount_of_illicit_predicted_transactions=}')
print(f'{amount_of_licit_predicted_transactions=}')


print(f'percent of illicit transactions')
x = ['illicit', 'licit']
y = [amount_of_illicit_predicted_transactions, amount_of_licit_predicted_transactions]
plt.bar(x, y)
# plt.text(x=0, y= 0,s= 0)
plt.text(x='illicit', y= amount_of_illicit_predicted_transactions * 1.1,s= amount_of_illicit_predicted_transactions)
plt.text(x='licit', y= amount_of_licit_predicted_transactions * 1.1,s= amount_of_licit_predicted_transactions)
plt.show()
